In [1]:
# The goal of this notebook is to create the database and tables for the metadata of the dataset
# This is necessary to be able to query the data more efficiently compared to the current method of reading the data from a large json file

In [2]:
from pycocotools.coco import COCO
from collections import defaultdict, Counter
from enum import IntEnum
import json

In [3]:
# Load the COCO dataset for ground truths
cocoGt = COCO('Objects365/test/zhiyuan_objv2_test.json')

loading annotations into memory...
Done (t=6.95s)
creating index...
index created!


In [4]:
# Count categories in each image
if cocoGt is not None:
    # Create a dictionary to count category occurrences per image
    img_obj = defaultdict(Counter)

    # Utilize the imgToAnns property to access annotations for each image
    for img_id, anns in cocoGt.imgToAnns.items():
        # Access all annotations for the current image
        for ann in anns:
            # Increment the count for this category
            img_obj[img_id][ann['category_id']] += 1

In [9]:
# Enumerate the Objects365 categories
class o365_cat(IntEnum):
    PERSON = 1
    CAT = 140
    DOG = 93
    PIGEON = 165
    PARROT = 320

# Filter the images based on the number of objects in each category
filtered_anns = dict()
for img_id, cnt in img_obj.items():
    # Skip images with more than 25 objects
    if sum(cnt.values()) > 25:
        continue
    
    # Single-element combinations
    if 1 < cnt[o365_cat.PERSON] <= 3 and cnt[o365_cat.CAT] == 0 and cnt[o365_cat.DOG] == 0 and (cnt[o365_cat.PIGEON] == 0 and cnt[o365_cat.PARROT] == 0):
        filtered_anns[img_id] = cnt # 1-3 people and no animals
    elif cnt[o365_cat.PERSON] == 0 and 1 < cnt[o365_cat.CAT] <= 3 and cnt[o365_cat.DOG] == 0 and (cnt[o365_cat.PIGEON] == 0 and cnt[o365_cat.PARROT] == 0):
        filtered_anns[img_id] = cnt # 1-3 cats and no other animals
    elif cnt[o365_cat.PERSON] == 0 and cnt[o365_cat.CAT] == 0 and 1 < cnt[o365_cat.DOG] <= 3 and (cnt[o365_cat.PIGEON] == 0 and cnt[o365_cat.PARROT] == 0):
        filtered_anns[img_id] = cnt # 1-3 dogs and no other animals
    elif cnt[o365_cat.PERSON] == 0 and cnt[o365_cat.CAT] == 0 and cnt[o365_cat.DOG] == 0 and 1 < cnt[o365_cat.PIGEON] + cnt[o365_cat.PARROT] <= 3:
        filtered_anns[img_id] = cnt # 1-3 birds and no other animals
    
    # Two-element combinations
    elif 1 < cnt[o365_cat.PERSON] <= 3 and 1 < cnt[o365_cat.CAT] <= 3 and cnt[o365_cat.DOG] == 0 and (cnt[o365_cat.PIGEON] == 0 and cnt[o365_cat.PARROT] == 0):
        filtered_anns[img_id] = cnt # 1-3 people and 1-3 cats and no other animals
    elif 1 < cnt[o365_cat.PERSON] <= 3 and cnt[o365_cat.CAT] == 0 and 1 < cnt[o365_cat.DOG] <= 3 and (cnt[o365_cat.PIGEON] == 0 and cnt[o365_cat.PARROT] == 0):
        filtered_anns[img_id] = cnt # 1-3 people and 1-3 dogs and no other animals
    elif 1 < cnt[o365_cat.PERSON] <= 3 and cnt[o365_cat.CAT] == 0 and cnt[o365_cat.DOG] == 0 and 1 < cnt[o365_cat.PIGEON] + cnt[o365_cat.PARROT] <= 3:
        filtered_anns[img_id] = cnt # 1-3 people and 1-3 birds and no other animals
    elif cnt[o365_cat.PERSON] == 0 and 1 < cnt[o365_cat.CAT] <= 3 and 1 < cnt[o365_cat.DOG] <= 3 and (cnt[o365_cat.PIGEON] == 0 and cnt[o365_cat.PARROT] == 0):
        filtered_anns[img_id] = cnt # 1-3 cats and 1-3 dogs and no other animals
    elif cnt[o365_cat.PERSON] == 0 and 1 < cnt[o365_cat.CAT] <= 3 and cnt[o365_cat.DOG] == 0 and 1 < cnt[o365_cat.PIGEON] + cnt[o365_cat.PARROT] <= 3:
        filtered_anns[img_id] = cnt # 1-3 cats and 1-3 birds and no other animals
    elif cnt[o365_cat.PERSON] == 0 and cnt[o365_cat.CAT] == 0 and 1 < cnt[o365_cat.DOG] <= 3 and 1 < cnt[o365_cat.PIGEON] + cnt[o365_cat.PARROT] <= 3:
        filtered_anns[img_id] = cnt # 1-3 dogs and 1-3 birds and no other animals
    
    # Three-element combinations
    elif 1 < cnt[o365_cat.PERSON] <= 3 and 1 < cnt[o365_cat.CAT] <= 3 and 1 < cnt[o365_cat.DOG] <= 3 and (cnt[o365_cat.PIGEON] == 0 and cnt[o365_cat.PARROT] == 0):
        filtered_anns[img_id] = cnt # 1-3 people and 1-3 cats and 1-3 dogs and no other animals
    elif 1 < cnt[o365_cat.PERSON] <= 3 and 1 < cnt[o365_cat.CAT] <= 3 and cnt[o365_cat.DOG] == 0 and 1 < cnt[o365_cat.PIGEON] + cnt[o365_cat.PARROT] <= 3:
        filtered_anns[img_id] = cnt # 1-3 people and 1-3 cats and 1-3 birds and no other animals
    elif 1 < cnt[o365_cat.PERSON] <= 3 and cnt[o365_cat.CAT] == 0 and 1 < cnt[o365_cat.DOG] <= 3 and 1 < cnt[o365_cat.PIGEON] + cnt[o365_cat.PARROT] <= 3:
        filtered_anns[img_id] = cnt # 1-3 people and 1-3 dogs and 1-3 birds and no other animals
    elif cnt[o365_cat.PERSON] == 0 and 1 < cnt[o365_cat.CAT] <= 3 and 1 < cnt[o365_cat.DOG] <= 3 and 1 < cnt[o365_cat.PIGEON] + cnt[o365_cat.PARROT] <= 3:
        filtered_anns[img_id] = cnt # 1-3 cats and 1-3 dogs and 1-3 birds and no other animals
    
    # Four-element combinations
    elif 1 < cnt[o365_cat.PERSON] <= 3 and 1 < cnt[o365_cat.CAT] <= 3 and 1 < cnt[o365_cat.DOG] <= 3 and 1 < cnt[o365_cat.PIGEON] + cnt[o365_cat.PARROT] <= 3:
        filtered_anns[img_id] = cnt # 1-3 people and 1-3 cats and 1-3 dogs and 1-3 birds

KeyboardInterrupt: 

In [6]:
# Enumerate the COCO categories
class coco_cat(IntEnum):
    PERSON = 0
    CAT = 15
    DOG = 16
    BIRD = 14

# Convert the filtered annotations to COCO format
filtered_coco = {
    'images': [],
    'annotations': [],
    'categories': []
}

for img_id, cnt in filtered_anns.items():
    # Access the annotations for the current image
    anns = cocoGt.imgToAnns[img_id]

    # Check for 'iscrowd' flag in any annotations; skip to next image if found
    if any(ann.get('iscrowd', 0) == 1 for ann in anns if ann['category_id'] in [cat.value for cat in o365_cat]):
        continue

    for ann in anns:
        # Check if the current annotation is in the filtered list
        if ann['category_id'] in [cat.value for cat in o365_cat]:
            # Create a new annotation for the current image
            new_ann = ann.copy()
            match new_ann['category_id']:
                case o365_cat.PERSON:
                    new_ann['category_id'] = coco_cat.PERSON
                case o365_cat.CAT:
                    new_ann['category_id'] = coco_cat.CAT
                case o365_cat.DOG:
                    new_ann['category_id'] = coco_cat.DOG
                case o365_cat.PIGEON:
                    new_ann['category_id'] = coco_cat.BIRD
                case o365_cat.PARROT:
                    new_ann['category_id'] = coco_cat.BIRD
            filtered_coco['annotations'].append(new_ann)

    # Add image info to the filtered COCO dataset
    filtered_coco['images'].append(cocoGt.imgs[img_id])
    
# Add the COCO categories to the filtered COCO dataset
filtered_coco['categories'].append({
    'id': coco_cat.PERSON,
    'name': 'person',
    'supercategory': 'person'
})
filtered_coco['categories'].append({
    'id': coco_cat.CAT,
    'name': 'cat',
    'supercategory': 'animal'
})
filtered_coco['categories'].append({
    'id': coco_cat.DOG,
    'name': 'dog',
    'supercategory': 'animal'
})
filtered_coco['categories'].append({
    'id': coco_cat.BIRD,
    'name': 'bird',
    'supercategory': 'animal'
})

In [7]:
with open('Objects365/test/filtered_data_coco.json', 'w') as f:
    json.dump(filtered_coco, f)

print(f"Images: {len(filtered_coco['images'])}")
print(f"Annotations: {len(filtered_coco['annotations'])}")
print(f"Categories: {len(filtered_coco['categories'])}")

Images: 38821
Annotations: 93127
Categories: 4
